In [2]:
from IPython.core.debugger import set_trace

import numpy as np 
from math import ceil, floor
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer

import nltk
nltk.download("punkt")
nltk.download('averaged_perceptron_tagger')
import re
from collections import Counter

from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from multiprocessing import cpu_count

import torch
from torch.nn import BCELoss, Module, Linear, AvgPool1d, MaxPool1d
from torch.nn.utils import clip_grad_norm_

import hdbscan
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
import seaborn as sns

from tqdm.notebook import tqdm
import time, logging, sys, json, os

%run -i "../news-topic-cls/core/models/base.py"
%run -i "../news-topic-cls/core/models/extension.py"
%run -i "../news-topic-cls/core/data/data.py"
%run -i "../news-topic-cls/core/utils/optim.py"
%run -i "../news-topic-cls/core/utils/utils.py"

[nltk_data] Downloading package punkt to /Users/bradloff/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/bradloff/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


No Apex Import


In [3]:
path_to_data = "../data/"

In [21]:
with open(path_to_data + "data_lm_devex.txt", "r") as file:
    lines = file.readlines()
    print(len(lines))

763180


In [24]:
with open(path_to_data + "data_lm_devex_mw.txt", "r") as file:
    lines = file.readlines()
    print(len(lines))

2075770


In [ ]:
filename = "test_data.csv"
if "test" in filename:
    df = pd.read_csv(path_to_data + filename).drop(labels="Unnamed: 0", axis=1)
else:
    df = pd.read_csv(path_to_data + filename)

In [ ]:
df.head()

In [ ]:
df, label_encoder = encode_labels(df)

abbrev_mapping = get_abbreveation_mapping(label_encoder)
df = binarize_labels(df, abbrev_mapping)
df.head()

In [ ]:
with open(path_to_data + "mongodb-dump18.json", "r") as mw_export:
    mw_export18 = json.load(mw_export)

with open(path_to_data + "mongodb-dump19.json", "r") as mw_export:
    mw_export19 = json.load(mw_export)

In [ ]:
print(mw_export18[0]["diffbot"]["text"])

In [ ]:
mw_export18[1]['development-relevance-classification']

In [ ]:
for export in mw_export18:
    if "diffbot" in export.keys():
        try:
            assert export['development-relevance-classification'][0]["tag_name"] == 'Humanitarian Development Related'
        except:
            print("Diffbot in Keys")
            print(export['development-relevance-classification'][0]["tag_name"])
    else:
        try:
            assert export['development-relevance-classification'][0]["tag_name"] == 'Not Humanitarian Development Related'
        except:
            print("Diffbot not in Keys")
            print(export['development-relevance-classification'][0]["tag_name"])

In [ ]:
article_mw1819 = pd.DataFrame(columns=["id", "type", "text", "categories"]) 

In [ ]:
all_mw_articles = []
for export in tqdm(mw_export18, total=len(mw_export18)):
    if "diffbot" in export.keys() and export['development-relevance-classification'][0]["tag_name"] == 'Humanitarian Development Related':
        text = export["diffbot"]["text"]
        all_mw_articles.append(text)

for export in tqdm(mw_export19, total=len(mw_export19)):
    if "diffbot" in export.keys() and export['development-relevance-classification'][0]["tag_name"] == 'Humanitarian Development Related':
        text = export["diffbot"]["text"]
        all_mw_articles.append(text)

In [ ]:
article_mw1819.text = all_mw_articles

In [ ]:
article_mw1819.to_csv("../data/article_meltwater1819.csv", index=False)

## Data for Language Model

In [33]:
config_name = "roberta-base"
transformer = TransformerOptions(config_name)

In [18]:
article_confirmed = pd.read_csv(path_to_data+"/article_confirmed.csv")
article_proposed = pd.read_csv(path_to_data+'/article_proposed.csv')
article_meltwater1819 = pd.read_csv(path_to_data+'/article_meltwater1819.csv')

In [19]:
articles_development = pd.concat([article_confirmed, article_proposed, article_meltwater1819])
articles_development.text = articles_development.text.astype(str)

In [20]:
print("Number Articles: {}".format(len(articles_development)))

Number Articles: 101015


In [23]:
articles_train, articles_eval = train_test_split(articles_development, test_size=0.1, random_state=0)

In [30]:
len(articles_train)

90913

In [31]:
len(articles_eval)

10102

In [26]:
text_train = articles_train.text.values
text_eval = articles_eval.text.values

In [27]:
open(os.path.join(path_to_data,'text_train.txt'), "w").write('\n\n'.join(text_train))
open(os.path.join(path_to_data,"text_eval.txt"), "w").write("\n\n".join(text_eval))

30177855